In [1]:
from utils import all_registers
from pprint import pprint
import list_comp as lc
import dict_transform as dt
import set_transform as st

# **Proyecto 3.1. Iteradores**
## Video Games Sales

#### Integrantes del equipo:
1.   Elorza Velásquez Margarita
2.   García González Axel Isaac
3.   García González Kevin

### **Utils: Corresponde con el modulo utils el cual se importó al inicio del documento**

> Devuelve una tupla que tiene una tupla con los headers del csv  y una tupla de tuplas con todos los registros y todas las columnas del archivo vgsales.csv

In [2]:
def all_registers(
        name_file: str) -> tuple[tuple[str, ...], tuple[tuple[str, ...], ...]]:
    '''Devuelve una tupla que tiene una tupla con los headers del csv 
    y una tupla de tuplas con todos los registros y todas las columnas del archivo vgsales.csv'''

    with open(name_file) as csvfile:
        games = csv.reader(csvfile, delimiter=',')
        h = tuple(next(games))

        tuple_games = tuple(tuple(game) for game in games)

        return (h, tuple_games)

### **Lists: Corresponde con el modulo list_comp el cual se importó al comienzo del documento como lc**

> *Modulo list_comp donde se encuentran declaradas las funciones para las listas*

In [3]:
from typing import Callable
import csv

> A continuación se muestra la función actividades.

In [4]:
def actividades(csvName) -> Callable:
    #-------------Primera accion del generador (Crear la lista de diccionarios de los juegos)--------------#

    lista = []
    with open(csvName) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            lista.append(row)

    print("-----------Lista Generada--------")
    yield lista


    #----------Actividad, de la lista extraida(del csv) solo dejar el diccionario con nombre, rango, plataforma, año, género---------#

    #Funcion que recibe un diccionario y regresa solo algunos elementos del mismo
    def quitarElementos(diccionario):
        return {'Rank':diccionario['Rank'], 'Name':diccionario['Name'],'Platform':diccionario['Platform'],'Genre':diccionario['Genre'],'Year':diccionario['Year']}

    print("-----------Actividad 1, quitar informacion--------")
    yield map(quitarElementos,lista)

    #----------Actividad, de la lista extraida(del csv) solo dejar los juegos de plataforma Wii

    print("-----------Actividad 2, dejar solo plataforma Wii--------")

    yield filter(lambda x : x['Platform'] == 'Wii',lista)

    #----------Actividad, de la lista extraida(del csv) filtrar por el año 2006 y solo mostrar el rango, el nombre y su género

    print("-----------Actividad 3, Filtrar por año 2006 y mostrar nombre,rango,genero--------")

    yield map(lambda x : {'Rank':x['Rank'],'Name':x['Name'],'Genre':x['Genre']},filter(lambda x : x['Year'] == '2006',lista))

    #----------Actividad, de la lista extraida(del csv) usar Generator comprehensions para tomar todos los juegos publicados por nintendo y sacar el promedio de Sales

    print("-----------Actividad 4, tomar juegos publicados por nintendo y sacar promedio de sus Sales")
    
    def sumaT(x):
        return float(x['NA_Sales']) + float(x['EU_Sales']) + float(x['JP_Sales']) + float(x['Other_Sales']) + float(x['Global_Sales'])

    yield ({'Name':i['Name'], 'Publisher':i['Publisher'] , 'Platform':['Platform'] , 'Sales' : round(sumaT(i)/5,2) } for i in lista if i['Publisher'] == 'Nintendo')

    #----------Actividad, de la lista extraida(del csv) con un generador ordenar los juegos por año de salida pero que se presenten del mas actual al mas viejo (filtrar los que no tengan año N/A ), que se muestre año, nombre y rango.

    print("-----------Actividad 5, Ordenar por año del mas nuevo al mas viejo mostrando  año, nombre y rango")

    yield ( {'Year':i['Year'] , 'Name':i['Name'] , 'Rank':i['Rank']} for i in reversed(sorted( filter( lambda x : x['Year'] != 'N/A' and x['Year'] != 'Adventure',lista), key = lambda x : x['Year'])) )

### **Dictionary: Corresponde con el modulo dict_transform el cual se importó al comienzo del documento como dt**

In [5]:
from statistics import mean
from functools import reduce

>Closure para obtener ciertas columnas de una tupla.

In [6]:
def select_columns(columns: list[str], h: tuple[str, ...]) -> Callable:

    def inner(data: tuple[str, ...]):
        return (data[h.index(c)] for c in columns)

    return inner

> 1. Diccionario con el 'Name' del juego como clave y la 'Platform' del juego como valor.

In [7]:
def first_dict(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> dict:

    # Map que recorre la lista y genera una tupla con el 'Name' y la 'Platform'
    # del juego para luego convertirlo en un diccionario.
    return dict(
        map(lambda game: (game[h.index("Name")], game[h.index("Platform")]),
            data))

> 2. Diccionario con el 'Name' del juego como clave y la 'Platform' del juego como valor, que sean del año 2009.

In [8]:
def second_dict(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> dict:

    # Map que recorre la lista y genera una tupla con el 'Name' y la 'Platform'
    # del juego de aquellos que son del año 2009 para luego convertirlo en un diccionario.
    return dict(
        map(lambda game: (game[h.index("Name")], game[h.index("Platform")]),
            filter(lambda game: game[h.index("Year")] == "2009", data)))

> 3. Diccionario con todos los registros que su 'Publisher' sea Sega

In [9]:
def third_dict(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> dict:

    # filter que filtra a los juegos que su 'Publisher' sea Sega
    filtered_games = filter(lambda game: game[h.index("Publisher")] == "Sega",
                            data)

    # Map que recorre la lista y genera una tupla con el 'Name' y un diccionario
    # con los datos del juego para luego convertirlo en un diccionario.
    return dict(
        map(
            lambda game: (game[h.index("Name")], {
                "Platform": game[h.index("Platform")],
                "Year": game[h.index("Year")],
                "Genre": game[h.index("Genre")],
                "Plublisher": game[h.index("Publisher")],
            }), filtered_games))

> 4. Diccionario que tiene el 'Name' del juego como clave y sus datos como valor y que 'Rank' sea mayor a 10 y menor a 20

In [10]:
def fourth_dict(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> dict:

    # filter que filtra a los juegos que su 'Rank' sea mayor a 10 y menor a 20
    filtered_games = filter(
        lambda game: int(game[h.index("Rank")]) > 10 and int(game[h.index(
            "Rank")]) < 20, data)

> 5. Diccionario con el 'Name' del juego como clave y un promedio de sus Sales como valor

In [11]:
def fifth_dict(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> dict:

    # Closure para obtener las columnas de ventas en diferentes territorios.
    sales_selected = select_columns(
        ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"], h)

    # Reduce que recorre la lista y genera un diccionario con el 'Name' del juego
    # como clave y un promedio de sus Sales como valor.
    return reduce(
        lambda dictionary, game: {
            **dictionary, game[h.index("Name")]:
            mean(map(float, sales_selected(game)))
        }, data, {})

### **Sets: Corresponde con el modulo set_transform el cual se importó al comienzo del documento como st**

> 1. Conjunto de los nombres de las plataformas

In [12]:
def first_set(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[str]:
    names = set(map(lambda x: x[h.index("Platform")], data))
    return names

> 2. Conjunto de todos los registros donde sus elementos sean la tupla 'Name' y 'Global_Sales' de los juegos

In [13]:
def second_set(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[tuple[str, str]]:
    get_data = set(map(lambda x: (x[h.index("Name")], x[h.index("Global_Sales")]), data))
    return get_data

> 3. Conjunto de todos los registros donde sus elementos sean la tupla 'Name' y 'NA_Sales' de los juegos que hayan vendido más de 5.0 millones en NA.

In [14]:
def third_set(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[tuple[str, str]]:
    keys = ('Name', 'NA_Sales')
    get_data = set(filter(lambda x: x[keys.index('NA_Sales')] > 5.0, map(lambda x: (x[h.index("Name")], float(x[h.index("NA_Sales")])), data)))
    return get_data

> 4. Conjunto de tuplas con las columnas: 'Name' y 'Global_Sales' de juegos donde su 'Publisher' sea Sega, su 'Platform' sea 3DS y que contengan la palabra Sonic.

In [15]:
def fourth_set(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[tuple[str, str]]:
    # keys = ['Name', 'Global_Sales']
    get_data = filter(lambda x: x[h.index('Publisher')] == "Sega" and x[h.index('Platform')] == "3DS" and x[h.index('Name')].find('Sonic') != -1,
                      data)
    get_filData = set(map(lambda x: (x[h.index('Name')], x[h.index('Global_Sales')]), get_data))
    
    return get_filData

> 5. Conjunto de los nombres de los 'Publisher' que poseen juegos del genero 'Platform' cuyo lanzamiento esté entre los años 2013 y 2016.

In [16]:
def fifth_set(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[str]:
    get_data = filter(lambda x: x[h.index('Genre')] == 'Platform' and x[h.index('Year')] >= '2013' and x[h.index('Year')] <= '2016', data)
    get_filData = set(map(lambda x: x[h.index('Publisher')], get_data))    
    return get_filData

> > *Para realizar las operaciones sobre conjuntos se definieron los siguientes conjuntos además de utilizar los ya obtenidos en la sección anterior.*

> Conjunto de tuplas con las columnas: Name y Global_Sales de juegos que su plublisher sea Sega y su plataforma 3DS.

In [17]:
def conjunto_p(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[tuple[str, str]]:
    get_data = filter(lambda x: x[h.index('Publisher')] == "Sega" and x[h.index('Platform')] == "3DS",data)
    get_filData = set(map(lambda x: (x[h.index('Name')], x[h.index('Global_Sales')]), get_data))
    
    return get_filData

> Conjunto de los nombres de las plataformas que tengan juegos del genero Action

In [18]:
def conjunto_q(data: tuple[tuple[str, ...], ...], h: tuple[str,...]) -> set[str]:
    get_data = filter(lambda x: x[h.index('Genre')] == 'Action', data)
    get_filData = set(map(lambda x: x[h.index('Platform')], get_data))
    return get_filData

> Conjunto de todos los registros que sus elementos sean la tupla "Name" y "NA_Sales" de los juegos que hayan vendido más de 4.0 millones en NA

In [19]:
def conjunto_u(data: tuple[tuple[str, ...], ...], h: tuple[str, ...]) -> set[tuple[str, str]]:
    keys = ('Name', 'NA_Sales')
    get_data = set(filter(lambda x: x[keys.index('NA_Sales')] > 4.0, map(lambda x: (x[h.index("Name")], float(x[h.index("NA_Sales")])), data)))
    return get_data

### **Main: Corresponde con el programa principal, aquí se realiza la ejecución de las funciones anteriores**

> 1. print_list, listas

In [20]:
def print_list() -> None:
    dir_csv = "./vgsales.csv"

    #Ciclo que sirve como iteradores encadenados y nos muestra las 5 actividades, genera una lista del CSV y musestra las 5 actividades puestas en el modulo List_comp.py
    for i in lc.actividades(dir_csv):

        #Comprueba que sea un iterador (Object)
        if (isinstance(i, object)):
            for j, k in enumerate(i):
                #Usando enumerate vamos a verificar que solo 3 se impriman, esto para no imprimir muchos juegos y puedan diferenciarse las actividades
                if j == 3:
                    print(k)
                    break
                print(k)

        # si no es un iterador es un iterable o lista
        else:
            print(i[:4])

> 2. print_dict, diccionarios

In [21]:
def print_dicts(h: tuple[str, ...], data: tuple[tuple[str, ...], ...]) -> None:
    print(
        "1. Diccionario con el 'Name' del juego como clave y la 'Platform' del juego como valor."
    )
    dict_one = dt.first_dict(data, h)
    pprint(dict(list(dict_one.items())[:3]))

    print(
        "\n2. Diccionario con el 'Name' del juego como clave y la 'Platform' del juego como valor que sean del año 2009."
    )
    dict_two = dt.second_dict(data, h)
    pprint(dict(list(dict_two.items())[:3]))

    print(
        "\n3. Diccionario con todos los registros que su 'Publisher' sea Sega")
    dict_three = dt.third_dict(data, h)
    pprint(dict(list(dict_three.items())[:3]))

    print(
        "\n4. Diccionario que tiene el 'Name' del juego como clave y sus datos como valor y que 'Rank' sea mayor a 10 y menor a 20"
    )
    dict_four = dt.fourth_dict(data, h)
    pprint(dict(list(dict_four.items())[:3]))

    print(
        "\n5. Diccionario con el 'Name' del juego como clave y un promedio de sus Sales como valor"
    )
    dict_five = dt.fifth_dict(data, h)
    pprint(dict(list(dict_five.items())[:3]))


> 3. print_set, conjuntos

In [22]:
def print_sets(h: tuple[str, ...], data: tuple[tuple[str, ...], ...]) -> None:
    print(
        "1. Conjunto de los nombres de las plataformas."
    )
    set_one = st.first_set(data, h)
    pprint(set(list(set_one)[:5]))
    
    print(
        "\n2. Conjunto de todos los registros que sus elementos sean la tupla "
        "'Name' y 'Global_Sales' de los juegos."
    )
    set_two = st.second_set(data, h)
    pprint(set(list(set_two)[:5]))

    print(
        "\n3. Conjunto de todos los registros donde sus elementos sean la tupla "
        "'Name' y 'NA_Sales' de los juegos que hayan vendido más de 5.0 "
        "millones en NA"
    )
    set_three = st.third_set(data, h)
    pprint(set(list(set_three)[:5]))

    print(
        "\n4. Conjunto de tuplas con las columnas: 'Name' y 'Global_Sales' de juegos "
        "donde su 'Publisher' sea Sega, su 'Platform' sea 3DS y que contengan la palabra Sonic"
    )
    set_four = st.fourth_set(data, h)
    pprint(set(list(set_four)[:5]))

    print(
        "\n5. Conjunto de los nombres de los 'Publisher' que poseen juegos del "
        "genero 'Platform' cuyo lanzamiento esté entre los años 2013 y "
        "2016"
    )
    set_five = st.fifth_set(data, h)
    pprint(set(list(set_five)[:5]))

> 4. print_set, operaciones con conjuntos

In [23]:
def print_operation_sets(h: tuple[str, ...], data: tuple[tuple[str, ...],...]) -> None:

    A = st.conjunto_p(data, h) - st.fourth_set(data, h)
    print(
        "\n1. Conjunto de las tuplas de los juegos donde su 'Platform' sea 3DS, "
        "su 'Publisher' sea Sega y su nombre no contenga la palabra Sonic."
    )
    pprint(set(list(A)[:5]))

    B = st.fifth_set(data, h).union(st.first_set(data, h))
    print(
        "\n2. Conjunto de los 'Publisher' que poseen juegos del genero 'Platform' "
        "cuyo lanzamiento esta entre los años 2013-2016 y los nombres de las plataformas."
    )
    pprint(set(list(B)[:5]))

    C = A.intersection(st.second_set(data, h))
    print(
        "\n3. Conjunto de tuplas que pertenezcan tanto al conjunto A "
           "y el resultado de ejercicio 2 de la sección anterior."
    )
    pprint(set(list(C)[:5]))

    D = st.conjunto_q(data, h) ^ st.first_set(data, h)
    print(
        "\n4. Conjunto complemento de las plataformas que pertenecen a "
           "los resultados del ejercicio 1 de la sección anterior "
           "y el conjunto de la llamada a la función conjunto_q()."
    )
    pprint(set(list(D)[:5]))

    E = st.third_set(data, h) & st.conjunto_u(data, h)
    print(
        "\n5. Conjunto de tuplas donde las ventas de los juegos "
         "que se encuentren en el conjunto del ejercicio 3 de la sección "
         "anterior y en el conjunto de la llamda a la función conjunto_u()."
    )
    pprint(set(list(E)[:5]))


In [24]:
h, data = all_registers('./vgsales.csv')

> 5. main, programa principal

In [25]:
# List
print("----------------------------- List  -------------------------------\n")
print_list()
print(
	"\n-------------------------------------------------------------------"
)

----------------------------- List  -------------------------------

-----------Lista Generada--------
{'Rank': '1', 'Name': 'Wii Sports', 'Platform': 'Wii', 'Year': '2006', 'Genre': 'Sports', 'Publisher': 'Nintendo', 'NA_Sales': '41.49', 'EU_Sales': '29.02', 'JP_Sales': '3.77', 'Other_Sales': '8.46', 'Global_Sales': '82.74'}
{'Rank': '2', 'Name': 'Super Mario Bros.', 'Platform': 'NES', 'Year': '1985', 'Genre': 'Platform', 'Publisher': 'Nintendo', 'NA_Sales': '29.08', 'EU_Sales': '3.58', 'JP_Sales': '6.81', 'Other_Sales': '0.77', 'Global_Sales': '40.24'}
{'Rank': '3', 'Name': 'Mario Kart Wii', 'Platform': 'Wii', 'Year': '2008', 'Genre': 'Racing', 'Publisher': 'Nintendo', 'NA_Sales': '15.85', 'EU_Sales': '12.88', 'JP_Sales': '3.79', 'Other_Sales': '3.31', 'Global_Sales': '35.82'}
{'Rank': '4', 'Name': 'Wii Sports Resort', 'Platform': 'Wii', 'Year': '2009', 'Genre': 'Sports', 'Publisher': 'Nintendo', 'NA_Sales': '15.75', 'EU_Sales': '11.01', 'JP_Sales': '3.28', 'Other_Sales': '2.96', 'Gl

In [26]:
# Dict
print("\n------------------------ Dictionary -------------------------------\n")
print_dicts(h, data)
print(
	"\n-------------------------------------------------------------------"
)


------------------------ Dictionary -------------------------------

1. Diccionario con el 'Name' del juego como clave y la 'Platform' del juego como valor.
{'Mario Kart Wii': 'Wii', 'Super Mario Bros.': 'GB', 'Wii Sports': 'Wii'}

2. Diccionario con el 'Name' del juego como clave y la 'Platform' del juego como valor que sean del año 2009.
{'New Super Mario Bros. Wii': 'Wii',
 'Wii Fit Plus': 'Wii',
 'Wii Sports Resort': 'Wii'}

3. Diccionario con todos los registros que su 'Publisher' sea Sega
{'Mario & Sonic at the Olympic Games': {'Genre': 'Sports',
                                        'Platform': 'DS',
                                        'Plublisher': 'Sega',
                                        'Year': '2008'},
 'Mario & Sonic at the Olympic Winter Games': {'Genre': 'Sports',
                                               'Platform': 'DS',
                                               'Plublisher': 'Sega',
                                               'Year': '2009'},

In [27]:
# Set
print("\n------------------------ Set -------------------------------\n")
print_sets(h, data)
print(
	"\n-------------------------------------------------------------------"
)


------------------------ Set -------------------------------

1. Conjunto de los nombres de las plataformas.
{'GB', 'XOne', 'PSV', '2010', '3DS'}

2. Conjunto de todos los registros que sus elementos sean la tupla 'Name' y 'Global_Sales' de los juegos.
{('Angry Birds Star Wars', '0.35'),
 ('Animal Crossing: City Folk', '4.68'),
 ('Hero Bank', '0.07'),
 ('Spider-Man: Shattered Dimensions', '0.45'),
 ('Suikoden Tactics', '0.2')}

3. Conjunto de todos los registros donde sus elementos sean la tupla 'Name' y 'NA_Sales' de los juegos que hayan vendido más de 5.0 millones en NA
{('Call of Duty: Black Ops 3', 5.77),
 ('Pokemon Black/Pokemon White', 5.57),
 ('Pokemon Red/Pokemon Blue', 11.27),
 ('Pokemon Ruby/Pokemon Sapphire', 6.06),
 ('Wii Sports Resort', 15.75)}

4. Conjunto de tuplas con las columnas: 'Name' y 'Global_Sales' de juegos donde su 'Publisher' sea Sega, su 'Platform' sea 3DS y que contengan la palabra Sonic
{('Mario & Sonic at the London 2012 Olympic Games', '1.19'),
 ('Sonic 

In [28]:
# Set Operations
print("\n------------------------ Set Opetarions -------------------------------\n")
print_operation_sets(h, data)
print(
	"\n-------------------------------------------------------------------"
)


------------------------ Set Opetarions -------------------------------


1. Conjunto de las tuplas de los juegos donde su 'Platform' sea 3DS, su 'Publisher' sea Sega y su nombre no contenga la palabra Sonic.
{('Captain America: Super Soldier', '0.08'),
 ('Hatsune Miku: Project Mirai 2', '0.22'),
 ('Hero Bank', '0.07'),
 ('Hero Bank 2', '0.05'),
 ('Puyo Puyo!! 20th Anniversary', '0.13')}

2. Conjunto de los 'Publisher' que poseen juegos del genero 'Platform' cuyo lanzamiento esta entre los años 2013-2016 y los nombres de las plataformas.
{'GB', 'XOne', 'PSV', 'Rain Games', 'Disney Interactive Studios'}

3. Conjunto de tuplas que pertenezcan tanto al conjunto A y el resultado de ejercicio 2 de la sección anterior.
{('Captain America: Super Soldier', '0.08'),
 ('Crush3D', '0.12'),
 ('Hero Bank', '0.07'),
 ('Hero Bank 2', '0.05'),
 ('Puyo Puyo!! 20th Anniversary', '0.13')}

4. Conjunto complemento de las plataformas que pertenecen a los resultados del ejercicio 1 de la sección anterior y